In [109]:
## found a way to remove white spances in columnsqq

import pandas as pd 
import numpy as np
import re

missing_values = ['', 'None','Notspecified']

df = pd.read_csv("all_sales_data.csv", na_values= missing_values)

# removing the spaces at the end and begining of the column name
for i in df.columns:
    df.rename(columns={i: i.strip()}, inplace=True)

# removes dataset from group 2
df = df.drop(df[df.source == 2].index)

# removing dolar signs, commas and euro signs in the price column
# df['price'] = df['price'].replace({'$': '', ',': '',"€":""}, regex=True)

# removing m² and white spaces
df['area'] = df['area'].replace({'m': '', ' ': '',"²":""}, regex=True)




In [86]:
df.area.isnull().sum()

5270

In [111]:
# area

df.area.fillna('-1', inplace=True)
df.loc[df.area == "Notspecified"] = np.nan
df['area'] = df['area'].str.replace('.0','')
df['area'] = df['area'].str.replace('0','-1')
df.loc[df['area'] == 0, 'area'] = '-1'
df.loc[df['area'] == '', 'area'] = '-1'
df.area.fillna(0, inplace=True)
df.area.isnull().sum()
df.area.unique()
# jf = df[pd.notnull(df.area)]
# jf[jf['area'].value.isnumeric()]






array(['242', '349', '3', '235', '2', '-1', '1', '156', '255', '137', '5',
       '118', '7', '232', '257', '265', '568', '927', '49', '585', '315',
       '259', '277', '325', '226', '245', '275', '175', '114', '352',
       '412', '519', '211', '268', '367', '123', '8', '125', '237', '177',
       '85', '361', '144', '128', '4', '648', '983', '184', '1262', '425',
       '423', '2-1', '195', '135', '187', '521', '93', '127', '9', '134',
       '224', '288', '446', '166', '178', '233', '358', '193', '115',
       '272', '387', '63', '189', '18', '172', '431', '171', '276', '23',
       '263', '159', '362', '331', '314', '133', '258', '332', '444',
       '136', '99', '165', '485', '112', '289', '221', '312', '264',
       '337', '231', '185', '354', '348', '222', '6', '498', '68', '313',
       '273', '427', '865', '461', '162', '62', '196', '153', '246',
       '262', '252', '248', '82', '169', '168', '145', '646', '627',
       '228', '236', '365', '143', '164', '562', '375', '518',

In [ ]:
#price
# must be higher than 10 000 

df[df.price.str.contains]